In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from io import StringIO

C:\Users\Chris\AppData\Local\Temp\ipykernel_30400\2900524488.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def scrape_batting_tables(url, html_text):
    print_url = 'https://baseball.pointstreak.com/' + url
    print(f"Accessing print page: {print_url}")
    print_response = requests.get(print_url)
    dfs = pd.read_html(StringIO(print_response.text), attrs={'border': '1'})
    if dfs:
        df = dfs[0]
        df.columns = df.iloc[0]
        df = df[1:]
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        df = df[df.player != 'Totals']
        soup = BeautifulSoup(html_text, 'html.parser')
        div = soup.find('div', style='width:900px')
        h2 = div.find('h2') if div else None
        title = h2.string if h2 else 'unknown'
        df_name = title.lower().replace(' ', '_') + '_df'
        globals()[df_name] = df
        print(f"Data stored in DataFrame: {df_name}")

def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    print_div = stats_soup.find('div', {'class': 'pull-left'})
                    print_link = print_div.find('a', href=True, string='Print')
                    if print_link is not None:
                        print_response = requests.get('https://baseball.pointstreak.com/' + print_link['href'])
                        scrape_batting_tables(print_link['href'], print_response.text)

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142674&seasonid=33653&view=batting
Data stored in DataFrame: edmonton_collegiate_batting_stats_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142669&seasonid=33653&view=batting
Data stored in DataFrame: okanagan_college_batting_stats_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142670&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/pr

In [4]:
victoria_collegiate_batting_stats_df.head()

,player,p,avg,g,ab,r,h,2b,3b,hr,...,ops,slg,obp,pa,ab/hr,bb/pa,bb/k,seca,isop,rc
1,"Burton, T",OF,.297,31,101,27,30,4,2,1,...,.806,.406,.400,123,101,.106,.813,.366,.109,15.465
2,"Deagle, R",IF,.291,30,110,20,32,8,0,3,...,.868,.445,.423,137,36.667,.153,.84,.345,.155,19.824
3,"Biello, D",OF,.290,26,100,26,29,5,1,1,...,.808,.390,.418,122,100,.148,1.5,.3,.1,15.534
4,"Green, B",IF,.283,29,113,23,32,2,1,0,...,.718,.319,.399,138,0,.138,1.462,.23,.035,13.909
5,"Harris, P",C,.263,23,76,18,20,1,1,2,...,.799,.382,.417,96,38,.167,1.067,.355,.118,11.348


# With Pitching

In [5]:
def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            print("Title at team page:", BeautifulSoup(team_response.text, 'html.parser').title.string)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    print("Title at stats page:", BeautifulSoup(stats_response.text, 'html.parser').title.string)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    div = stats_soup.find('div', style='margin-bottom:15px;')
                    if div is not None:
                        pitching_link = div.find('a', href=True, string='Pitching')
                        if pitching_link is not None:
                            pitching_url = 'https://baseball.pointstreak.com/' + pitching_link['href']
                            print(f"Accessing pitching page: {pitching_url}")
                            pitching_response = requests.get(pitching_url)
                            print("Title at pitching page:", BeautifulSoup(pitching_response.text, 'html.parser').title.string)
                            # Call scrape_batting_tables or a similar function for pitching data here

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Title at team page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team home | Pointstreak Sports Technologies
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Title at stats page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team stats | Pointstreak Sports Technologies
Accessing pitching page: https://baseball.pointstreak.com/?teamid=142674&seasonid=33653&view=pitching
Title at pitching page: Edmonton Collegiate -  | Pointstreak Sports Technologies
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Title at team page: Okanagan College - Canadian College Baseball Conference (CCBC) - team home | Pointstreak Sports Technologies
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Title at stats page: Okana

In [2]:
def scrape_tables(stat_type, url, html_text):
    print_url = 'https://baseball.pointstreak.com/' + url
    print(f"Accessing {stat_type} page: {print_url}")
    print_response = requests.get(print_url)
    dfs = pd.read_html(StringIO(print_response.text), attrs={'border': '1'})
    if dfs:
        df = dfs[0]
        df.columns = df.iloc[0]
        df = df[1:]
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        df = df[df.player != 'Totals']
        soup = BeautifulSoup(html_text, 'html.parser')
        div = soup.find('div', style='width:900px')
        h2 = div.find('h2') if div else None
        title = h2.string if h2 else 'unknown'
        df_name = title.lower().replace(' ', '_') + '_df'
        globals()[df_name] = df
        print(f"Data stored in DataFrame: {df_name}")
def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            print("Title at team page:", BeautifulSoup(team_response.text, 'html.parser').title.string)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    print("Title at stats page:", BeautifulSoup(stats_response.text, 'html.parser').title.string)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    div = stats_soup.find('div', style='margin-bottom:15px;')
                    if div is not None:
                        pitching_link = div.find('a', href=True, string='Pitching')
                        if pitching_link is not None:
                            pitching_url = 'https://baseball.pointstreak.com/' + pitching_link['href']
                            print(f"Accessing pitching page: {pitching_url}")
                            pitching_response = requests.get(pitching_url)
                            print("Title at pitching page:", BeautifulSoup(pitching_response.text, 'html.parser').title.string)
                            scrape_tables('pitching', pitching_link['href'], pitching_response.text)

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Title at team page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team home | Pointstreak Sports Technologies
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Title at stats page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team stats | Pointstreak Sports Technologies
Accessing pitching page: https://baseball.pointstreak.com/?teamid=142674&seasonid=33653&view=pitching
Title at pitching page: Edmonton Collegiate -  | Pointstreak Sports Technologies
Accessing pitching page: https://baseball.pointstreak.com/?teamid=142674&seasonid=33653&view=pitching


ValueError: No tables found